In [ ]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Satria Data/dataset_penyisihan_bdc_2024.csv", delimiter=';')
df.head()

,text,label
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik


In [ ]:
df['text'][3]

'RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7Q= Sewaktu anies bersikap kritis ke kinerja pak prabowo dianggap engga sopan karena dianggap kurang menghormati orang tua, giliran skrg gibran yg tengil dan sok kritis malah dianggap kritis dan keras. Apakah ini tidak standar ganda? Ã°ÂŸÂ¤Â\xad [RE coldthem]'

In [ ]:
def preprocessing_data(text):
    text = re.sub(r'RT\s|@\S+|\[RE\s\w+\]|http\S+|[^a-zA-Z0-9\s]', '', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [ ]:
# praproses data
df['clean_data'] = df['text'].apply(preprocessing_data)
# df['label'] = df['label'].astype('category')
df.head()

,text,label,clean_data
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam,kunjungan prabowo ini untuk meresmikan dan men...
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik,anies dapat tepuk tangan meriah saat jadi rekt...
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi,emng bener sih pendukung 01 ada yg goblok begi...
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik,sewaktu anies bersikap kritis ke kinerja pak p...
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik,anies baswedan harap asn termasuk tni dan polr...


In [ ]:
kata_singkat = {
    'yang': ['yg'],
    'orang': ['org', 'org2', 'orang2'],
    'sebagai': ['sbg', 'sbgai', 'sbgmn'],
    'kepada': ['kpd'],
    'terima kasih': ['trm ksh'],
    'akan': ['akn'],
    'untuk': ['utk'],
    'dengan': ['dg', 'dgn'],
    'yang lain': ['yl'],
    'benar-benar': ['benar2'],
    'tidak': ['tdk', "gak", "ga", "enggak", "engga", 'gk'],
    'gimana': ['gmna'],
    'kamu': ['km'],
    'politik': ['pol'],
    'olahraga': ['olah raga'],
    'seperti': ['spt', 'sprti', "sperti"],
    'anak': ['anakanak', 'anak2', "ank", "ank2"],
    'karena': ['krn', "karna"],
    'lalu': ['ll'],
    'tanggung jawab': ['tjwb'],
    'menggunakan': ['mggn'],
    'sudah': ['sdh'],
    'dalam': ['dlm'],
    'melakukan': ['mllk'],
    'tidak ada': ['tdk ada'],
    'sehingga': ['sgg'],
    'tersebut': ['tsb', "trsbut"],
    'semua': ['smua'],
    'mengapa': ['knp'],
    'saat': ['st'],
    'kapan': ['kpn'],
    'sekali': ['skli', "skl"],
    'mungkin': ['mngkn'],
    'sangat': ['sgt'],
    'sampai': ['smpt'],
    'tetapi': ['tp'],
    "negara": ["ngri", "negri", "ng"],
    "presiden": ["pres", 'pr3siden', "prs", 'presidn'],
    "kampanye": ["kampenye"],
    "bagaimana": ["bgmna", "gmn"],
    "masyarakat": ["msy", "msyrkt", "msyrt"],
    "kita": ["kta"],
    "sekarang": ["skrg", "skr"],
    "mereka": ["mrk"],
    "tahun": ["thn", "thn2", 'tahun2', 'th'],
    'ke': ['ke3', 'ke2', 'ke101', 'ke33', 'ke45'],
    'dikarang': ['dikarang2'],
    'cara': ['cara2'],
    'kata': ['kata2'],
    'penguin': ['penguin2'],
    'benteng': ['benteng2'],
    'ekonomi': ['3konomi'],
    'kejadian': ['kejadian2'],
    'siber': ['sib3r'],
    'ketua': ['k3tua', "ktu"],
    'menang': ['m3nang'],
    'lawan': ['lawan2'],
    'anggota': ['anggota2'],
    'abal': ['abal2'],
    'abu': ['abu2'],
    'teman': ['teman2'],
    'sama': ['sama2'],
    'ri': ['ri1'],
    'hal': ['hal2'],
    'ancem': ['ancem2'],
    'sedikit': ['sedikit2'],
    'joged': ['joged2'],
    'kelompok': ['kelompok2'],
    'tuntutan': ['tuntutan2'],
    'istimewa': ['istim3wa'],
    'omon': ['omon3'],
    'mengejar': ['mengejar2'],
    'tenggelamkan': ['t3nggelamkan'],
    'preman': ['preman2'],
    'coblos': ['coblos3'],
    'bertanya': ['bertanya2'],
    'cuap': ['cuap2'],
    'bareng': ['bar3ng'],
    'geopolitik': ['g3opolitik'],
    'berhadapan': ['berhadap2an'],
    'internet': ['int3rnet'],
    "bangsa": ["bgs"],
    "begitu": ["bgt"],
    "bapak": ["pak", 'bpk'],
    "berhasil": ["brhasil"],
    "berapa": ["brp"],
    "bisa": ["bs", 'bsa'],
    "bukan": ["bkn"],
    "capres": ["cpres"],
    "cawapres": ["cwpres"],
    "contek": ["ctk"],
    "dan": ["dn"],
    "desa": ["dsa"],
    "diberi": ["dberi"],
    "dirindukan": ["drindukan"],
    "dosen": ["dsn"],
    "foto": ["fto"],
    "ganjar": ["gjr"],
    "gara-gara": ["grgr"],
    "gubernur": ["gub"],
    "halal": ["hl"],
    "hanya": ["hny"],
    "indonesia": ["ind"],
    "ini": ["ni"],
    "jalur": ["jl"],
    "jawa": ["jw"],
    "jokowi": ["jkw"],
    "jurusan": ["jrs"],
    "kader": ["kdr"],
    "kalau": ["kl"],
    "kalian": ["kln"],
    "kanan": ["kn"],
    "keterangan": ["ktr"],
    "lahan": ["lh"],
    "mahfud": ["mf"],
    "menjadi": ["mj"],
    "mobilisasi": ["mbl"],
    "mengharapkan": ["mhrpkn"],
    "mengingatkan": ["mgngatkn"],
    "mengkritik": ["mgkritk"],
    "menteri": ["mtr", "mtrny"],
    "merupakan": ["mrpk"],
    "mulai": ["ml"],
    "murid": ["mrd"],
    "partai": ["prt"],
    "pemimpin": ["pmpin"],
    "pendidikan": ["pndidikan"],
    "pertama": ["prtm"],
    "rakyat": ["rky", "rykt"],
    "senang": ["snang"],
    "tanggung": ["tnggung"],
    "tempat": ["tmpt"],
    "undang-undang": ["u-u", 'uu'],
    "yaitu": ["yitu"],
    "orang": ["org", "org2"],
    "calon presiden": ["capres"],
    "calon wakil presiden": ["cawapres"],
    "kemudian": ["kmd"],
    "sebab": ["sb"],
    "jangan": ["jgn"],
    "dari": ["dr"],
    "saya": ["sy"],
    "warga": ["wrg"],
    "keluarga": ["klg"],
    "nomor": ["no", 'no1'],
    "negeri": ["negr", "negri"],
    'pdip': ['pdip1'],
    'rp': ['rp0'],
    'bawa': ['bawa2'],
    'kayu': ['kayu2'],
    'dp': ['dp0'],
    'pulau': ['11pulau', '10pulau']
}

In [ ]:
def ganti_kata(teks, kata_singkat):
    pattern = re.compile(r'\b(' + '|'.join(re.escape(kata) for sublist in kata_singkat.values() for kata in sublist) + r')\b')

    def replace(match):
        kata = match.group(0)
        for key, values in kata_singkat.items():
            if kata in values:
                return key
        return kata

    hasil = [pattern.sub(replace, kalimat) for kalimat in teks]
    return hasil

In [ ]:
df['clean_data2'] = ganti_kata(df['clean_data'], kata_singkat)
# df['label'] = df['label'].astype('category')
df.head()

,text,label,clean_data,clean_data2
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam,kunjungan prabowo ini untuk meresmikan dan men...,kunjungan prabowo ini untuk meresmikan dan men...
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik,anies dapat tepuk tangan meriah saat jadi rekt...,anies dapat tepuk tangan meriah saat jadi rekt...
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi,emng bener sih pendukung 01 ada yg goblok begi...,emng bener sih pendukung 01 ada yang goblok be...
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik,sewaktu anies bersikap kritis ke kinerja pak p...,sewaktu anies bersikap kritis ke kinerja bapak...
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik,anies baswedan harap asn termasuk tni dan polr...,anies baswedan harap asn termasuk tni dan polr...


In [ ]:
# hapus data kosong
whitespace_rows = df[df['clean_data2']==""]
count_whitespace_rows = len(whitespace_rows)
df = df[df['clean_data2'].notna() & (df['clean_data2'] != '')]

In [ ]:
df.shape

(4991, 4)

In [ ]:
pip install indoNLP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
from indoNLP.preprocessing import replace_slang

df['no_slang'] = df['clean_data2'].apply(replace_slang)

<ipython-input-16-1a078b2201eb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['no_slang'] = df['clean_data2'].apply(replace_slang)


In [ ]:
from indoNLP.preprocessing import replace_word_elongation

df['no_elong'] = df['no_slang'].apply(replace_word_elongation)

<ipython-input-18-1644663ca0d3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['no_elong'] = df['no_slang'].apply(replace_word_elongation)


In [ ]:
# REMOVE STOPWORD
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')

f = open("/content/drive/MyDrive/Satria Data/list_stopword_ind.txt", "r")
stopword_id = []
for line in f:
    stripped_line = line.strip()
    line_list = stripped_line.split()
    stopword_id.append(line_list[0])
f.close()

len(stopword_id)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


764

In [ ]:
stop_words = set(stopwords.words('english'))
combined_stop_words = stop_words.union(set(stopword_id))

def remove_stopwords(text):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in combined_stop_words]
    return ' '.join(filtered_words)

df['nostopword_text'] = df['no_elong'].apply(remove_stopwords)
df.head()

<ipython-input-21-c1e679cc788c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nostopword_text'] = df['no_elong'].apply(remove_stopwords)


,text,label,clean_data,clean_data2,no_slang,no_elong,nostopword_text
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam,kunjungan prabowo ini untuk meresmikan dan men...,kunjungan prabowo ini untuk meresmikan dan men...,kunjungan prabowo ini untuk meresmikan dan men...,kunjungan prabowo ini untuk meresmikan dan men...,kunjungan prabowo meresmikan menyerahkan proye...
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik,anies dapat tepuk tangan meriah saat jadi rekt...,anies dapat tepuk tangan meriah saat jadi rekt...,anies dapat tepuk tangan meriah saat jadi rekt...,anies dapat tepuk tangan meriah saat jadi rekt...,anies tepuk tangan meriah rektor mewajibkan ma...
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi,emng bener sih pendukung 01 ada yg goblok begi...,emng bener sih pendukung 01 ada yang goblok be...,memang benar sih pendukung 01 ada yang goblok ...,memang benar sih pendukung 01 ada yang goblok ...,sih pendukung 01 goblok pendukung 02 ridwan ka...
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik,sewaktu anies bersikap kritis ke kinerja pak p...,sewaktu anies bersikap kritis ke kinerja bapak...,sewaktu anies bersikap kritis ke kinerja bapak...,sewaktu anies bersikap kritis ke kinerja bapak...,anies bersikap kritis kinerja prabowo dianggap...
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik,anies baswedan harap asn termasuk tni dan polr...,anies baswedan harap asn termasuk tni dan polr...,anies baswedan harap asn termasuk tni dan polr...,anies baswedan harap asn termasuk tni dan polr...,anies baswedan harap asn tni polri pegang sump...


In [ ]:
!pip install imblearn sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.1 MB/s eta 0:00:00


In [ ]:
# STEMMING
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming_text(text):
  return stemmer.stem(text)

In [ ]:
df['stemming_nostopword_text'] = df['nostopword_text'].apply(stemming_text)
df.tail()

<ipython-input-24-57023de945e8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['stemming_nostopword_text'] = df['nostopword_text'].apply(stemming_text)


,text,label,clean_data,clean_data2,no_slang,no_elong,nostopword_text,stemming_nostopword_text
4995,RT @l1DsGGe2xObT3t72dHwqlT58X7jvEYtEnauZIZSYwS...,Politik,ngeliat debat kemaren pas prabowo kicep kekira...,ngeliat debat kemaren pas prabowo kicep kekira...,melihat debat kemarin pas prabowo kicep kekira...,melihat debat kemarin pas prabowo kicep kekira...,debat kemarin pas prabowo kicep kekira keluarg...,debat kemarin pas prabowo kicep kira keluarga ...
4996,Masyarakat yakin bahwa Prabowo-Gibran memiliki...,Politik,masyarakat yakin bahwa prabowogibran memiliki ...,masyarakat yakin bahwa prabowogibran memiliki ...,masyarakat yakin bahwa prabowogibran memiliki ...,masyarakat yakin bahwa prabowogibran memiliki ...,masyarakat prabowogibran memiliki visi sejalan...,masyarakat prabowogibran milik visi jalan aspi...
4997,imo both are irrational but yg satu jauh lebih...,Ekonomi,imo both are irrational but yg satu jauh lebih...,imo both are irrational but yang satu jauh leb...,imo both are irrational but yang satu jauh leb...,imo both are irrational but yang satu jauh leb...,imo irrational irrational tbh nevertheles duan...,imo irrational irrational tbh nevertheles dua ...
4998,@cIIGSdjaPrmAQY1E4gWnLpIZSdyQn8ZMhjJzgOsxfRM= ...,Pertahanan dan Keamanan,look at that pak ganjar anda sdh berkecimpung ...,look at that bapak ganjar anda sudah berkecimp...,look at that bapak ganjar anda sudah berkecimp...,look at that bapak ganjar anda sudah berkecimp...,look ganjar berkecimpung lgislatif eksekutif b...,look ganjar kecimpung lgislatif eksekutif bhka...
4999,"Acara ini tidak hanya memasak, tetapi Calon Pr...",Sumber Daya Alam,acara ini tidak hanya memasak tetapi calon pre...,acara ini tidak hanya memasak tetapi calon pre...,acara ini tidak hanya memasak tetapi calon pre...,acara ini tidak hanya memasak tetapi calon pre...,acara memasak calon presiden nomor urut 2 meng...,acara masak calon presiden nomor urut 2 habis ...


In [ ]:
df.head()

,text,label,clean_data,clean_data2,no_slang,no_elong,nostopword_text,stemming_nostopword_text
0,Kunjungan Prabowo ini untuk meresmikan dan men...,Sumber Daya Alam,kunjungan prabowo ini untuk meresmikan dan men...,kunjungan prabowo ini untuk meresmikan dan men...,kunjungan prabowo ini untuk meresmikan dan men...,kunjungan prabowo ini untuk meresmikan dan men...,kunjungan prabowo meresmikan menyerahkan proye...,kunjung prabowo resmi serah proyek bantu air b...
1,RT Anies dapat tepuk tangan meriah saat jadi R...,Politik,anies dapat tepuk tangan meriah saat jadi rekt...,anies dapat tepuk tangan meriah saat jadi rekt...,anies dapat tepuk tangan meriah saat jadi rekt...,anies dapat tepuk tangan meriah saat jadi rekt...,anies tepuk tangan meriah rektor mewajibkan ma...,anies tepuk tangan riah rektor wajib mata kuli...
2,@CIqXqwGAT04tMtx4OCATxjoVq7vv/Y8HeYaIOgMFg8Y= ...,Demografi,emng bener sih pendukung 01 ada yg goblok begi...,emng bener sih pendukung 01 ada yang goblok be...,memang benar sih pendukung 01 ada yang goblok ...,memang benar sih pendukung 01 ada yang goblok ...,sih pendukung 01 goblok pendukung 02 ridwan ka...,sih dukung 01 goblok dukung 02 ridwan kamil sk...
3,RT @L3R8XFBw3WGbxRPSj0/0hHZTbqVGX7qtfwRg9zmhK7...,Politik,sewaktu anies bersikap kritis ke kinerja pak p...,sewaktu anies bersikap kritis ke kinerja bapak...,sewaktu anies bersikap kritis ke kinerja bapak...,sewaktu anies bersikap kritis ke kinerja bapak...,anies bersikap kritis kinerja prabowo dianggap...,anies sikap kritis kerja prabowo anggap sopan ...
4,Anies Baswedan Harap ASN termasuk TNI dan Polr...,Politik,anies baswedan harap asn termasuk tni dan polr...,anies baswedan harap asn termasuk tni dan polr...,anies baswedan harap asn termasuk tni dan polr...,anies baswedan harap asn termasuk tni dan polr...,anies baswedan harap asn tni polri pegang sump...,anies baswedan harap asn tni polri pegang sump...


In [ ]:
df = df.drop(columns=['text', 'clean_data', 'clean_data2', 'nostopword_text', 'no_slang', 'no_elong'], axis =1)
df.head()

,label,stemming_nostopword_text
0,Sumber Daya Alam,kunjung prabowo resmi serah proyek bantu air b...
1,Politik,anies tepuk tangan riah rektor wajib mata kuli...
2,Demografi,sih dukung 01 goblok dukung 02 ridwan kamil sk...
3,Politik,anies sikap kritis kerja prabowo anggap sopan ...
4,Politik,anies baswedan harap asn tni polri pegang sump...


In [ ]:
df = df.rename(columns={'stemming_nostopword_text' : 'text_clean'})

In [ ]:
df.to_csv('satria_data_clean.csv', index=False)